In [154]:
import numpy as np
import pandas as pd
import time

In [155]:
start_time=time.time()

In [156]:
data = pd.read_csv(r"C:\Users\GiuliaChiaretti\Desktop\foundations\googleplaystore.csv", sep = ',')
data_user_review = pd.read_csv(r"C:\Users\GiuliaChiaretti\Desktop\foundations\googleplaystore_user_reviews.csv", sep=',')
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [157]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
App               10841 non-null object
Category          10841 non-null object
Rating            9367 non-null float64
Reviews           10841 non-null object
Size              10841 non-null object
Installs          10841 non-null object
Type              10840 non-null object
Price             10841 non-null object
Content Rating    10840 non-null object
Genres            10841 non-null object
Last Updated      10841 non-null object
Current Ver       10833 non-null object
Android Ver       10838 non-null object
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [158]:
data_user_review.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [159]:
data_user_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64295 entries, 0 to 64294
Data columns (total 5 columns):
App                       64295 non-null object
Translated_Review         37427 non-null object
Sentiment                 37432 non-null object
Sentiment_Polarity        37432 non-null float64
Sentiment_Subjectivity    37432 non-null float64
dtypes: float64(2), object(3)
memory usage: 2.5+ MB


In [160]:
data_user_review.isnull().sum()

App                           0
Translated_Review         26868
Sentiment                 26863
Sentiment_Polarity        26863
Sentiment_Subjectivity    26863
dtype: int64

Si sceglie di eliminare la riga numero 10472 in quanto non possedendo la variabile 'Categoty' potrebbe creare problemi nel corso dell'analisi.

In [161]:
data.loc[[10472]]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [162]:
data = data.drop(10472)

In [163]:
data.shape

(10840, 13)

### 1) Convert the app sizes to a number

Dopo aver verificato l'assenza di missing value nella variabile 'Size' si procede nel seguente modo:
 - si crea una nuova variabile NewSize. In questo modo, non modificando la variabile originaria 'Size' sarà possibile confrontarle.
 - per tutti le app in cui 'Size' = "Varies with device" la nuova variabile 'NewSize' sarà missing.
 - si rimuovono le lettere 'k' e 'M' e si trasformano i valori da object a float.
 - si moltiplicano i valori ottenuti per 1000 o 1000000, in base alla presenza rispettivamente di 'k' o 'M' nella variabile originaria 'Size'

Infine, si effetuno alcuni check.

In [164]:
data['Size'].unique()

array(['19M', '14M', '8.7M', '25M', '2.8M', '5.6M', '29M', '33M', '3.1M',
       '28M', '12M', '20M', '21M', '37M', '2.7M', '5.5M', '17M', '39M',
       '31M', '4.2M', '7.0M', '23M', '6.0M', '6.1M', '4.6M', '9.2M',
       '5.2M', '11M', '24M', 'Varies with device', '9.4M', '15M', '10M',
       '1.2M', '26M', '8.0M', '7.9M', '56M', '57M', '35M', '54M', '201k',
       '3.6M', '5.7M', '8.6M', '2.4M', '27M', '2.5M', '16M', '3.4M',
       '8.9M', '3.9M', '2.9M', '38M', '32M', '5.4M', '18M', '1.1M',
       '2.2M', '4.5M', '9.8M', '52M', '9.0M', '6.7M', '30M', '2.6M',
       '7.1M', '3.7M', '22M', '7.4M', '6.4M', '3.2M', '8.2M', '9.9M',
       '4.9M', '9.5M', '5.0M', '5.9M', '13M', '73M', '6.8M', '3.5M',
       '4.0M', '2.3M', '7.2M', '2.1M', '42M', '7.3M', '9.1M', '55M',
       '23k', '6.5M', '1.5M', '7.5M', '51M', '41M', '48M', '8.5M', '46M',
       '8.3M', '4.3M', '4.7M', '3.3M', '40M', '7.8M', '8.8M', '6.6M',
       '5.1M', '61M', '66M', '79k', '8.4M', '118k', '44M', '695k', '1.6M',
     

In [165]:
data['Size'].isnull().sum()

0

In [166]:
data['NewSize'] = (data[data.Size != "Varies with device"].Size.replace(r'[kM]+$', '', regex=True).astype(float) *
    data[data.Size != "Varies with device"].Size.str.extract(r'[\d\.]+([kM]+)', expand=False).replace(['k','M'], [10**3, 10**6]).astype(int))

In [167]:
data[['Size','NewSize']]

,Size,NewSize
0,19M,19000000.0
1,14M,14000000.0
2,8.7M,8700000.0
3,25M,25000000.0
4,2.8M,2800000.0
...,...,...
10836,53M,53000000.0
10837,3.6M,3600000.0
10838,9.5M,9500000.0
10839,Varies with device,NaN


In [168]:
data.NewSize.value_counts().head()

11000000.0    198
12000000.0    196
14000000.0    194
13000000.0    191
15000000.0    184
Name: NewSize, dtype: int64

In [169]:
data.NewSize.isnull().sum()

1695

In [170]:
data.Size.value_counts().head(6)

Varies with device    1695
11M                    198
12M                    196
14M                    194
13M                    191
15M                    184
Name: Size, dtype: int64

### 2) Convert the number of installs to a number

Inizialmente si osserva la struttura della varibile 'Installs'. Successivamente si crea una nuova variabile 'NewInstalls' in cui sono stati rimossi tutti i caratteri non numerici tramite il comando replace di pandas e la funzione regex ' \ \\ D' ed, in seguito, ne è stato modificato il type da object a numerico.

In [171]:
data.Installs.unique()

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '50+', '100+', '500+',
       '10+', '1+', '5+', '0+', '0'], dtype=object)

In [172]:
data['NewInstalls'] = (data.Installs.replace('\\D', '', regex=True).astype(float))

In [173]:
data.NewInstalls.unique()

array([1.e+04, 5.e+05, 5.e+06, 5.e+07, 1.e+05, 5.e+04, 1.e+06, 1.e+07,
       5.e+03, 1.e+08, 1.e+09, 1.e+03, 5.e+08, 5.e+01, 1.e+02, 5.e+02,
       1.e+01, 1.e+00, 5.e+00, 0.e+00])

### 3) Transform “Varies with device” into a missing value

Dopo aver trasformato in missing value le osservazioni per cui la variabile 'Size' era pari a 'Varies with device', si effettua un check sfruttando la nuova variabile creata al punto 1. 
Nella creazione della variabile 'NewSize', infatti, le osservazioni per cui 'Size' era pari a 'Varies with device' erano già state etichettate come NaN. Si verifica, quindi, di avere lo stesso numero di missing value in 'Size' e 'NewSize'.


In [174]:
data['Size'].isnull().sum()

0

In [175]:
data.Size[40:45]

40                   15M
41                   10M
42    Varies with device
43                  1.2M
44                   12M
Name: Size, dtype: object

In [176]:
data = data.replace(to_replace="Varies with device", value = np.nan)

In [177]:
data['NewSize'].isnull().sum()

1695

In [178]:
data['Size'].isnull().sum()

1695

Come si può vedere, dopo aver sostituito tutti i 'Varies with device' con dei missing value, il numero di missing nella vecchia variabile 'Size' e nella nuova 'NewSize' è in entrambi i casi giustamente uguale a 1695.
Si effettua un'ulteriore verifica osservando lo stesso campione di 5 osservazioni stampato prcedentemente per vedere come è cambiata l'osservazione nella riga 42.

In [179]:
data.Size[40:45]

40     15M
41     10M
42     NaN
43    1.2M
44     12M
Name: Size, dtype: object

### 4) Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

Si utilizzano i comandi di regex per attuare un replace dei caratteri non numerici sia nella variabile 'Current Ver' che 'Android Ver'. 

In [180]:
data[['Android Ver','Current Ver']].head(20)

,Android Ver,Current Ver
0,4.0.3 and up,1.0.0
1,4.0.3 and up,2.0.0
2,4.0.3 and up,1.2.4
3,4.2 and up,NaN
4,4.4 and up,1.1
5,2.3 and up,1.0
6,4.0.3 and up,1.1
7,4.2 and up,6.1.61.1
8,3.0 and up,2.9.2
9,4.0.3 and up,2.8


In [181]:
data['Current Ver'].unique()

array(['1.0.0', '2.0.0', '1.2.4', ..., '1.0.612928', '0.3.4', '2.0.148.0'],
      dtype=object)

In [182]:
data['Android Ver'].unique()

array(['4.0.3 and up', '4.2 and up', '4.4 and up', '2.3 and up',
       '3.0 and up', '4.1 and up', '4.0 and up', '2.3.3 and up', nan,
       '2.2 and up', '5.0 and up', '6.0 and up', '1.6 and up',
       '1.5 and up', '2.1 and up', '7.0 and up', '5.1 and up',
       '4.3 and up', '4.0.3 - 7.1.1', '2.0 and up', '3.2 and up',
       '4.4W and up', '7.1 and up', '7.0 - 7.1.1', '8.0 and up',
       '5.0 - 8.0', '3.1 and up', '2.0.1 and up', '4.1 - 7.1.1',
       '5.0 - 6.0', '1.0 and up', '2.2 - 7.1.1', '5.0 - 7.1.1'],
      dtype=object)

In [183]:
data["Current Ver"].replace("[a-z]+|[A-z]+", value = "", regex = True, inplace = True) 
data['Android Ver'].replace("[a-z]+|[A-z]+", value = "", regex = True, inplace = True) 

In [184]:
data['Android Ver'].unique()

array(['4.0.3  ', '4.2  ', '4.4  ', '2.3  ', '3.0  ', '4.1  ', '4.0  ',
       '2.3.3  ', nan, '2.2  ', '5.0  ', '6.0  ', '1.6  ', '1.5  ',
       '2.1  ', '7.0  ', '5.1  ', '4.3  ', '4.0.3 - 7.1.1', '2.0  ',
       '3.2  ', '7.1  ', '7.0 - 7.1.1', '8.0  ', '5.0 - 8.0', '3.1  ',
       '2.0.1  ', '4.1 - 7.1.1', '5.0 - 6.0', '1.0  ', '2.2 - 7.1.1',
       '5.0 - 7.1.1'], dtype=object)

In [185]:
data[['Android Ver','Current Ver']].head(20)

,Android Ver,Current Ver
0,4.0.3,1.0.0
1,4.0.3,2.0.0
2,4.0.3,1.2.4
3,4.2,NaN
4,4.4,1.1
5,2.3,1.0
6,4.0.3,1.1
7,4.2,6.1.61.1
8,3.0,2.9.2
9,4.0.3,2.8


### 5) Remove the duplicates

Si considera come duplicata una riga riferita alla stessa App. Con i seguenti comandi si verifica che sono presenti 1181 righe duplicate. 

Per decidere quale delle righe duplicate eliminare, si sceglie di tenere in considerazione il numero di review e conservare, per ogni App, la riga con il numero di review più alto, ovvero la più aggiornata e più recente in termini cronologici.


In [186]:
data.shape[0] - len(np.unique(data['App']))

1181

In [187]:
data['Reviews'].dtype #verifica del type della variabile 'Reviews'

dtype('O')

In [188]:
data['Reviews'] = data['Reviews'].astype(int)

In [189]:
data['Reviews'].dtype

dtype('int32')

Dopo aver trasformato la variabile 'Reviews' in numerica, si ordina il dataset in modo decrescente in base al numero di reviews e si conserva solamente la prima riga per ogni app, ovvero la riga con il numero di reviews più alto.
Infine, si verifica che dopo il processo di rimozione, il numero di duplicati sia zero e si mostrano le nuove dimesioni del dataset.

In [190]:
data = data.sort_values('Reviews', ascending = False) 
data = data.drop_duplicates(subset = 'App',keep='first') 

In [191]:
data.shape[0] - len(np.unique(data['App'])) 

0

In [192]:
data.shape

(9659, 15)

### 6) For each category, compute the number of apps

In [193]:
data.groupby(['Category'])['App'].count()

Category
ART_AND_DESIGN           61
AUTO_AND_VEHICLES        85
BEAUTY                   53
BOOKS_AND_REFERENCE     222
BUSINESS                420
COMICS                   56
COMMUNICATION           315
DATING                  170
EDUCATION               107
ENTERTAINMENT            87
EVENTS                   64
FAMILY                 1875
FINANCE                 345
FOOD_AND_DRINK          112
GAME                    945
HEALTH_AND_FITNESS      288
HOUSE_AND_HOME           73
LIBRARIES_AND_DEMO       84
LIFESTYLE               369
MAPS_AND_NAVIGATION     131
MEDICAL                 395
NEWS_AND_MAGAZINES      254
PARENTING                60
PERSONALIZATION         376
PHOTOGRAPHY             281
PRODUCTIVITY            374
SHOPPING                202
SOCIAL                  239
SPORTS                  325
TOOLS                   829
TRAVEL_AND_LOCAL        219
VIDEO_PLAYERS           164
WEATHER                  79
Name: App, dtype: int64

### 7) For each category, compute the average rating

In [194]:
data.groupby(['Category'])['Rating'].mean()

Category
ART_AND_DESIGN         4.359322
AUTO_AND_VEHICLES      4.190411
BEAUTY                 4.278571
BOOKS_AND_REFERENCE    4.344970
BUSINESS               4.098479
COMICS                 4.181481
COMMUNICATION          4.121484
DATING                 3.980451
EDUCATION              4.354717
ENTERTAINMENT          4.129885
EVENTS                 4.435556
FAMILY                 4.183576
FINANCE                4.115563
FOOD_AND_DRINK         4.171277
GAME                   4.244432
HEALTH_AND_FITNESS     4.243033
HOUSE_AND_HOME         4.140984
LIBRARIES_AND_DEMO     4.178125
LIFESTYLE              4.093355
MAPS_AND_NAVIGATION    4.036441
MEDICAL                4.165862
NEWS_AND_MAGAZINES     4.121569
PARENTING              4.300000
PERSONALIZATION        4.332215
PHOTOGRAPHY            4.155894
PRODUCTIVITY           4.183389
SHOPPING               4.230556
SOCIAL                 4.247291
SPORTS                 4.216154
TOOLS                  4.040278
TRAVEL_AND_LOCAL       4.069519

### 8) Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

Per svolgere questo task si creano:
- 'genres': una tabella contenente i generi. Come si può vedere, alcune app risultavano avere due generi diferenti, separati da ';'. Per questo questa nuova tabella genres contiene, per ognuna delle 9659 app, due colonne "FirstGenre" e "SecondGenre" (quest'ultima colonna risulta 'None' se l'app è associata ad un solo genere).
- apps: una tabella contenente le app
- app-genre: tabella derivante dal merge delle due precedenti. E' una tabella a 3 colonne: la prima contenente il nome dell'app, la seconda che specifica se si tratta del primo o secondo genere e la terza che riporta il nome del genere associatovi. A questo punto vengono create per ogni app (variabile utilizzata come id) due righe, per cui si verifica che questa tabella abbia dimensioni doppie rispetto alle precedenti.
    Successivamente, vengono eliminate le righe in cui il secondo genere risultava nullo per cui si ottiene un dataset finale in cui l'app compare una volta se associata ad un solo genere, due se associata a due generi.


In [195]:
data.Genres.unique()

array(['Social', 'Communication', 'Strategy', 'Tools', 'Arcade',
       'Video Players & Editors', 'Casual', 'Sports', 'News & Magazines',
       'Action', 'Photography', 'Adventure', 'Casual;Action & Adventure',
       'Travel & Local', 'Racing', 'Personalization', 'Maps & Navigation',
       'Entertainment', 'Trivia', 'Education;Education', 'Shopping',
       'Productivity', 'Health & Fitness', 'Adventure;Action & Adventure',
       'Simulation', 'Books & Reference', 'Lifestyle', 'Role Playing',
       'Arcade;Action & Adventure', 'Weather', 'Puzzle', 'Casino', 'Card',
       'Word', 'Puzzle;Action & Adventure', 'Sports;Action & Adventure',
       'Board', 'Finance', 'Puzzle;Brain Games',
       'Racing;Action & Adventure', 'Business', 'Education',
       'Food & Drink', 'Educational;Pretend Play', 'Comics',
       'Card;Action & Adventure', 'Music',
       'Educational;Action & Adventure', 'Parenting',
       'Simulation;Action & Adventure', 'Role Playing;Action & Adventure',
      

In [196]:
genres= data['Genres'].str.split(';', expand = True)
genres=genres.rename(columns = {0: 'FirstGenre', 1:'SecondGenre'})
genres.head()

,FirstGenre,SecondGenre
2544,Social,None
381,Communication,None
2604,Social,None
382,Communication,None
1879,Strategy,None


In [197]:
genres[20:30]

,FirstGenre,SecondGenre
4676,Communication,None
7937,Action,None
2884,Photography,None
474,Communication,None
1885,Casual,None
468,Communication,None
1764,Adventure,None
2090,Casual,Action & Adventure
1893,Casual,None
3975,Casual,None


In [198]:
genres.shape

(9659, 2)

In [199]:
apps = pd.DataFrame(data.App)

In [200]:
app_genre = pd.merge(apps, genres, left_index = True, right_index = True).melt(id_vars = ['App'], value_name = "Genres")

In [201]:
app_genre.shape #19318 = 9659*2

(19318, 3)

In [202]:
app_genre.head()

,App,variable,Genres
0,Facebook,FirstGenre,Social
1,WhatsApp Messenger,FirstGenre,Communication
2,Instagram,FirstGenre,Social
3,Messenger – Text and Video Chat for Free,FirstGenre,Communication
4,Clash of Clans,FirstGenre,Strategy


Come verifica si stampano i dati relativi a 10 app. 

In [203]:
app_genre.loc[20:30,:]

,App,variable,Genres
20,Viber Messenger,FirstGenre,Communication
21,Shadow Fight 2,FirstGenre,Action
22,Google Photos,FirstGenre,Photography
23,LINE: Free Calls & Messages,FirstGenre,Communication
24,Pou,FirstGenre,Casual
25,Skype - free IM & video calls,FirstGenre,Communication
26,Pokémon GO,FirstGenre,Adventure
27,Minion Rush: Despicable Me Official Game,FirstGenre,Casual
28,Yes day,FirstGenre,Casual
29,Hay Day,FirstGenre,Casual


In [204]:
app_genre.loc[9679:9689,:]

,App,variable,Genres
9679,Viber Messenger,SecondGenre,None
9680,Shadow Fight 2,SecondGenre,None
9681,Google Photos,SecondGenre,None
9682,LINE: Free Calls & Messages,SecondGenre,None
9683,Pou,SecondGenre,None
9684,Skype - free IM & video calls,SecondGenre,None
9685,Pokémon GO,SecondGenre,None
9686,Minion Rush: Despicable Me Official Game,SecondGenre,Action & Adventure
9687,Yes day,SecondGenre,None
9688,Hay Day,SecondGenre,None


In [205]:
app_genre = app_genre.drop(columns = "variable").dropna().reset_index(drop = True)
app_genre.loc[20:30,:]

,App,Genres
20,Viber Messenger,Communication
21,Shadow Fight 2,Action
22,Google Photos,Photography
23,LINE: Free Calls & Messages,Communication
24,Pou,Casual
25,Skype - free IM & video calls,Communication
26,Pokémon GO,Adventure
27,Minion Rush: Despicable Me Official Game,Casual
28,Yes day,Casual
29,Hay Day,Casual


In [206]:
app_genre.loc[9679:9689,:]

,App,Genres
9679,Sandbox - Color by Number Coloring Pages,Creativity
9680,Bubbu – My Virtual Pet,Pretend Play
9681,Equestria Girls,Action & Adventure
9682,Plants vs. Zombies™ Heroes,Action & Adventure
9683,Hello Kitty Nail Salon,Pretend Play
9684,Animal Jam - Play Wild!,Pretend Play
9685,Real Racing 3,Action & Adventure
9686,MyScript Calculator,Education
9687,Barbie Magical Fashion,Creativity
9688,Masha and the Bear Child Games,Education


In [207]:
app_genre.shape 

(10051, 2)

In [208]:
print ('Numero di app che hanno associato un secondo genere:', app_genre.shape[0] - apps.shape[0])

Numero di app che hanno associato un secondo genere: 392


In [209]:
app_genre[app_genre.App == "Pixel Draw - Number Art Coloring Book"]

,App,Genres
4831,Pixel Draw - Number Art Coloring Book,Art & Design
9953,Pixel Draw - Number Art Coloring Book,Creativity


### 9) For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

Vengono costruite tramite la funzione get_dummies di pandas due tabelle:
- Genre1 è composta da 9659 righe (app) e 48 colonne, una per ogni genere associato alle app come primo.
- Genre2 è composta da 9659 righe (app) e 6 colonne, che sono i 6 generi associati come secondi ad alcune app.

Successivamente, queste 54 colonne vengono aggiunte al dataset iniziale.


In [210]:
Genre1= pd.get_dummies(genres['FirstGenre'], dtype = 'bool')

In [211]:
Genre2= pd.get_dummies(genres['SecondGenre'], dtype = 'bool')

In [212]:
Genre1.shape

(9659, 48)

In [213]:
Genre1.head(1)

,Action,Adventure,Arcade,Art & Design,Auto & Vehicles,Beauty,Board,Books & Reference,Business,Card,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
2544,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [214]:
Genre2.shape 

(9659, 6)

In [215]:
data.shape

(9659, 15)

In [216]:
data = pd.concat([data, Genre1], axis = 1)
data = pd.concat([data, Genre2], axis = 1)
data.shape

(9659, 69)

In [217]:
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Trivia,Video Players & Editors,Weather,Word,Action & Adventure,Brain Games,Creativity,Education,Music & Video,Pretend Play
2544,Facebook,SOCIAL,4.1,78158306,NaN,"1,000,000,000+",Free,0,Teen,Social,...,False,False,False,False,False,False,False,False,False,False
381,WhatsApp Messenger,COMMUNICATION,4.4,69119316,NaN,"1,000,000,000+",Free,0,Everyone,Communication,...,False,False,False,False,False,False,False,False,False,False
2604,Instagram,SOCIAL,4.5,66577446,NaN,"1,000,000,000+",Free,0,Teen,Social,...,False,False,False,False,False,False,False,False,False,False
382,Messenger – Text and Video Chat for Free,COMMUNICATION,4.0,56646578,NaN,"1,000,000,000+",Free,0,Everyone,Communication,...,False,False,False,False,False,False,False,False,False,False
1879,Clash of Clans,GAME,4.6,44893888,98M,"100,000,000+",Free,0,Everyone 10+,Strategy,...,False,False,False,False,False,False,False,False,False,False


In [218]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9659 entries, 2544 to 5086
Data columns (total 69 columns):
App                        9659 non-null object
Category                   9659 non-null object
Rating                     8196 non-null float64
Reviews                    9659 non-null int32
Size                       8431 non-null object
Installs                   9659 non-null object
Type                       9658 non-null object
Price                      9659 non-null object
Content Rating             9659 non-null object
Genres                     9659 non-null object
Last Updated               9659 non-null object
Current Ver                8596 non-null object
Android Ver                8666 non-null object
NewSize                    8431 non-null float64
NewInstalls                9659 non-null float64
Action                     9659 non-null bool
Adventure                  9659 non-null bool
Arcade                     9659 non-null bool
Art & Design               965

### 10) For each genre, compute the average rating. What is the genre with highest average?

Per calcolare il punteggio per ogni genere viene costruita una tabella avente 3 colonne: 'App', 'Genres', 'Rating'.
Per evitare problematiche derivanti dalle app con doppio genere, si usilizza la tabella precedentemente creata 'app_genre', in cui se un app è associata a due generi compare due volte. Questa tabella viene mergiata con il dataset iniziale in modo da ottenere le informazioni sul 'Rating'. 
Infine, viene calcolato il punteggio per ogni genere e vengono mostrati i 5 più alti.

In [219]:
app_genre_rating =  app_genre.merge(data, on = 'App').loc[:,['App','Genres_x','Rating']]
app_genre_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10051 entries, 0 to 10050
Data columns (total 3 columns):
App         10051 non-null object
Genres_x    10051 non-null object
Rating      8574 non-null float64
dtypes: float64(1), object(2)
memory usage: 314.1+ KB


In [220]:
app_genre_rating.groupby('Genres_x')[['Rating']].mean().sort_values(by = 'Rating', ascending = False).head(5)

,Rating
Genres_x,
Events,4.435556
Puzzle,4.370732
Brain Games,4.358065
Art & Design,4.350000
Books & Reference,4.343275


### 11) For each app, compute the approximate income, obtain as a product of number of installs and price.

Dopo aver rimosso il carattere '$' presente nella variabile 'Price', si converte in numerica e si crea la nuova variabile 'Income'. Successivamente si mostrano le 5 app con un 'Income' più alto e le 5 con l'income più basso, che ovviamente risultano essere tra quelle gratuite ('Price'=0).

In [221]:
data.NewInstalls.unique()

array([1.e+09, 1.e+08, 5.e+08, 5.e+07, 1.e+07, 5.e+06, 1.e+06, 5.e+05,
       1.e+05, 5.e+04, 1.e+04, 5.e+03, 1.e+03, 5.e+02, 1.e+02, 1.e+01,
       5.e+01, 5.e+00, 1.e+00, 0.e+00])

In [222]:
data.Price.unique()

array(['0', '$6.99', '$0.99', '$2.99', '$1.99', '$2.49', '$4.99', '$5.99',
       '$4.49', '$9.99', '$3.99', '$1.49', '$3.95', '$7.99', '$3.49',
       '$8.99', '$13.99', '$19.99', '$11.99', '$12.99', '$2.90', '$17.99',
       '$399.99', '$29.99', '$14.99', '$2.95', '$4.77', '$24.99', '$3.90',
       '$2.50', '$3.28', '$1.20', '$2.59', '$9.00', '$1.59', '$1.00',
       '$5.49', '$18.99', '$299.99', '$1.97', '$400.00', '$16.99',
       '$389.99', '$33.99', '$10.00', '$10.99', '$4.84', '$37.99',
       '$1.61', '$8.49', '$4.60', '$79.99', '$4.29', '$1.70', '$1.50',
       '$1.29', '$19.40', '$3.08', '$379.99', '$2.56', '$15.46', '$7.49',
       '$4.59', '$2.00', '$14.00', '$6.49', '$15.99', '$74.99', '$3.88',
       '$3.02', '$39.99', '$89.99', '$5.00', '$1.75', '$1.26', '$2.60',
       '$19.90', '$4.80', '$1.76', '$46.99', '$3.04', '$4.85', '$30.99',
       '$3.61', '$154.99', '$394.99', '$109.99', '$1.96', '$1.04',
       '$28.99', '$25.99', '$200.00'], dtype=object)

In [223]:
data['NewPrice'] = data['Price'].str.replace('$','',regex=True).astype(float)

In [224]:
data['Income'] = data['NewInstalls'] * data['NewPrice']


In [225]:
data[['App','Income','Price','NewInstalls']].sort_values('Income',ascending = False).head(5)

,App,Income,Price,NewInstalls
2241,Minecraft,69900000.0,$6.99,10000000.0
5351,I am rich,39999000.0,$399.99,100000.0
5356,I Am Rich Premium,19999500.0,$399.99,50000.0
4034,Hitman Sniper,9900000.0,$0.99,10000000.0
7417,Grand Theft Auto: San Andreas,6990000.0,$6.99,1000000.0


In [226]:
data[['App','Income','Price','NewInstalls']].sort_values('Income',ascending = False).tail(5)

,App,Income,Price,NewInstalls
55,Tickets SDA 2018 and Exam from the State Traff...,0.0,0,100000.0
1884,Super Jim Jump - pixel 3d,0.0,0,1000000.0
9783,Ankle Surgery ER Emergency,0.0,0,1000000.0
2137,Cute Pet Puppies,0.0,0,1000000.0
5086,Apps for SportsBєtting.ag - Bitcoin Welcome here!,0.0,0,1000.0


### 12) For each app, compute its minimum and maximum: Sentiment_polarity

In [227]:
data_user_review.groupby(['App'])['Sentiment_Polarity'].aggregate({'min','max'}).head(10)

,max,min
App,,
10 Best Foods for You,1.000000,-0.800000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000,-0.112500
11st,1.000000,-1.000000
1800 Contacts - Lens Store,0.838542,-0.300000
1LINE – One Line with One Touch,1.000000,-0.825000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,1.000000,-0.800000
21-Day Meditation Experience,0.587500,-0.265625
"2Date Dating App, Love and matching",1.000000,-0.645833
2GIS: directory & navigator,1.000000,-0.375000


In [228]:
print('Total execution time:', time.time()-start_time, 'seconds')

Total execution time: 1.4160442352294922 seconds
